In [1]:
# Import Libraries
import xarray as xr
import pandas as pd
import numpy as np
import scipy.signal
import datetime as dt
import metpy 

# Define Functions 
  1. removeSC - removes seasonal cycle
  2. detrend - removes the linear trend from data
  3. calStdNorAnom - Calculates normalised anomalies i.e. multiyear monthly anomaly (using sliding window of 31 years centred at that particular year) is devided by mulyiyear monthly standard deviation (using sliding window of 31 years centred at that particular year). Note - For initial (last) 15 years, values are taken as mean/standard deviation of first (last) 31  years is calculated.

In [2]:
def remove_time_mean(x):
    return x - x.mean(dim='time')

def removeSC(x):
    return x.groupby('time.month').apply(remove_time_mean)
    
def detrend(x,dim):
    y=x.copy()
    y.values=scipy.signal.detrend(x,dim)
    return y

# Calculate std normal anomaly
def calStdNorAnom(x):
    a=[]
    for m in np.unique(x.time.dt.month):
        mData=x[x.time.dt.month==m]
        mRolling=mData.rolling(time=31, center=True).mean().bfill(dim="time").ffill(dim="time")
        sRolling=mData.rolling(time=31, center=True).std().bfill(dim="time").ffill(dim="time")
        normData=(mData-mRolling)/sRolling
        a.append(normData)
    combineArray=xr.concat(a,'time')
    outArray=combineArray.sortby('time')
    return outArray


# CMIP6 Files

In [3]:
# Open Inout files
rsut_ds   = xr.open_dataset("../CMIP6nc/CESM2_historical_r1i1p1f1_rsut.nc")
rsdt_ds   = xr.open_dataset("../CMIP6nc/CESM2_historical_r1i1p1f1_rsdt.nc")
rsutcs_ds = xr.open_dataset("../CMIP6nc/CESM2_historical_r1i1p1f1_rsutcs.nc")
clivi_ds  = xr.open_dataset("../CMIP6nc/CESM2_historical_r1i1p1f1_clivi.nc")
clwvi_ds  = xr.open_dataset("../CMIP6nc/CESM2_historical_r1i1p1f1_clwvi.nc")
clt_ds    = xr.open_dataset("../CMIP6nc/CESM2_historical_r1i1p1f1_clt.nc")
psl_ds    = xr.open_dataset("../CMIP6nc/CESM2_historical_r1i1p1f1_psl.nc")
pr_ds     = xr.open_dataset("../CMIP6nc/CESM2_historical_r1i1p1f1_pr.nc")
tas_ds    = xr.open_dataset("../CMIP6nc/CESM2_historical_r1i1p1f1_tas.nc")


# Input Variables

In [4]:
## Calculate CRES
cres=(rsdt_ds.rsdt - rsut_ds.rsut) - (rsdt_ds.rsdt - rsutcs_ds.rsutcs)
### Preprocessing
### remove Seasonal Cycle
cres_RSC=removeSC(cres)
### Detrend
cres_RSC_dt=detrend(cres_RSC,0)
## Calculate Standard normal anomaly 
FinalOut=calStdNorAnom(cres_RSC_dt)
cres_Pre_ds = xr.Dataset({'cres': (('time','lat','lon'), FinalOut.data, {'long_name':'Cloud radiative effect in shortwave', 'units' :'W m-2', 'Description':'(rsdt - rsut) - (rsdt - rsutcs)'})}, coords={'time': cres.time,'lat': cres.lat,'lon': cres.lon})
cres_Pre_ds.cres


<xarray.DataArray 'cres' (time: 1980, lat: 192, lon: 288)>
array([[[-0.5362606 , -0.53626055, -0.536286  , ..., -0.53624594,
         -0.5362871 , -0.53655523],
        [-0.19532053, -0.21685699, -0.2256854 , ..., -0.36194375,
         -0.33467382, -0.24888033],
        [-0.9965259 , -1.007365  , -1.075501  , ..., -1.1434942 ,
         -1.1448454 , -1.0356848 ],
        ...,
        [-1.6770456 , -1.6770463 , -1.6770486 , ..., -1.6770464 ,
         -1.6770494 , -1.6770539 ],
        [-1.6770468 , -1.6770545 , -1.6770488 , ..., -1.677049  ,
         -1.6770521 , -1.6770461 ],
        [-1.6770544 , -1.6770511 , -1.6770569 , ..., -1.6770556 ,
         -1.6770529 , -1.677051  ]],

       [[ 0.4872965 ,  0.48729527,  0.48728636, ...,  0.4873001 ,
          0.487297  ,  0.48730332],
        [ 0.66292214,  0.67421204,  0.6982703 , ...,  0.5985639 ,
          0.61075944,  0.61749154],
        [ 0.6803259 ,  0.6833522 ,  0.6937574 , ...,  0.6032875 ,
          0.6217246 ,  0.64037323],
...
        [ 1.6768936 ,  1.6768878 ,  1.6769845 , ...,  1.6769712 ,
          1.67719   ,  1.6772577 ],
        [ 1.6769266 ,  1.6770521 ,  1.6768059 , ...,  1.6769696 ,
          1.6771814 ,  1.6769091 ],
        [ 1.6770784 ,  1.6771052 ,  1.6771955 , ...,  1.6772218 ,
          1.6772351 ,  1.6770686 ]],

       [[ 0.3493584 ,  0.34948182,  0.34948197, ...,  0.3494539 ,
          0.3493606 ,  0.34938055],
        [ 0.17763327,  0.20400424,  0.20153397, ...,  0.22626857,
          0.20665583,  0.21971244],
        [ 0.03690965,  0.04705519, -0.03912109, ...,  0.12287149,
          0.0943765 ,  0.06729995],
        ...,
        [ 1.6768751 ,  1.6768936 ,  1.6769547 , ...,  1.6769788 ,
          1.677195  ,  1.6772625 ],
        [ 1.6769364 ,  1.6770186 ,  1.6768003 , ...,  1.6769618 ,
          1.6771756 ,  1.6768829 ],
        [ 1.6770545 ,  1.6770971 ,  1.6772203 , ...,  1.6771914 ,
          1.6772466 ,  1.6770712 ]]], dtype=float32)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:    Cloud radiative effect in shortwave
    units:        W m-2
    Description:  (rsdt - rsut) - (rsdt - rsutcs)

In [5]:
## Calculate alpha
alpha= rsut_ds.rsut/rsdt_ds.rsdt
## replace NaN by 0 
alpha=alpha.fillna(0)
### Preprocessing
### remove Seasonal Cycle
alpha_RSC=removeSC(alpha)
### Detrend
alpha_RSC_dt=detrend(alpha_RSC,0)
## Calculate Standard normal anomaly 
FinalOut=calStdNorAnom(alpha_RSC_dt)
alpha_Pre_ds = xr.Dataset({'alpha': (('time','lat','lon'), FinalOut.data, {'long_name':'Planetary Albedo', 'Description':' (rsut/rsdt)'})}, coords={'time': alpha.time,'lat': alpha.lat,'lon': alpha.lon})
alpha_Pre_ds.alpha

<xarray.DataArray 'alpha' (time: 1980, lat: 192, lon: 288)>
array([[[ 0.9286614 ,  0.9286328 ,  0.9286036 , ...,  0.9285799 ,
          0.9286438 ,  0.9291648 ],
        [ 0.67326516,  0.6925612 ,  0.69187576, ...,  0.80949646,
          0.79086614,  0.7231903 ],
        [ 1.2564014 ,  1.2471325 ,  1.2911232 , ...,  1.3879788 ,
          1.375004  ,  1.2897283 ],
        ...,
        [-1.6770486 , -1.677046  , -1.6770487 , ..., -1.6770469 ,
         -1.6770467 , -1.6770455 ],
        [-1.6770493 , -1.6770526 , -1.6770501 , ..., -1.6770548 ,
         -1.6770451 , -1.6770496 ],
        [ 1.6770493 ,  1.6770523 ,  1.6770455 , ...,  1.6770495 ,
          1.6770464 ,  1.6770474 ]],

       [[-0.00524902, -0.00525409, -0.00522654, ..., -0.00545175,
         -0.00544129, -0.00537435],
        [-0.05765169, -0.07143366, -0.10206284, ...,  0.00343749,
         -0.0081493 , -0.01423705],
        [-0.22108592, -0.24651736, -0.2871537 , ..., -0.14227398,
         -0.15066427, -0.17008694],
...
        [ 1.6770794 ,  1.6770154 ,  1.676904  , ...,  1.6769342 ,
          1.676873  ,  1.6769811 ],
        [ 1.677104  ,  1.6769992 ,  1.677069  , ...,  1.6771731 ,
          1.6769179 ,  1.6771178 ],
        [-1.6769265 , -1.676978  , -1.676881  , ..., -1.6770287 ,
         -1.676855  , -1.6769135 ]],

       [[-1.4704679 , -1.4707795 , -1.4707202 , ..., -1.470771  ,
         -1.470534  , -1.4704903 ],
        [-1.3194141 , -1.3477223 , -1.34293   , ..., -1.348369  ,
         -1.3329306 , -1.3494536 ],
        [-1.1809788 , -1.194183  , -1.1220636 , ..., -1.1783092 ,
         -1.1928191 , -1.192973  ],
        ...,
        [ 1.677077  ,  1.6770185 ,  1.6769098 , ...,  1.6769484 ,
          1.6768701 ,  1.6769979 ],
        [ 1.6770922 ,  1.6769913 ,  1.6770661 , ...,  1.6771603 ,
          1.6769165 ,  1.677122  ],
        [-1.6769242 , -1.6770066 , -1.6768764 , ..., -1.6770325 ,
         -1.6768727 , -1.6769197 ]]], dtype=float32)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:    Planetary Albedo
    Description:   (rsut/rsdt)

In [6]:
##  clivi
clivi=  clivi_ds.clivi
### Preprocessing
### remove Seasonal Cycle
clivi_RSC=removeSC(clivi)
### Detrend
clivi_RSC_dt=detrend(clivi_RSC,0)
## Calculate Standard normal anomaly 
FinalOut=calStdNorAnom(clivi_RSC_dt)
clivi_Pre_ds = xr.Dataset({'clivi': (('time','lat','lon'), FinalOut.data, {'long_name':clivi.long_name, 'units':clivi.units})}, coords={'time': clivi.time,'lat': clivi.lat,'lon': clivi.lon})
clivi_Pre_ds.clivi

<xarray.DataArray 'clivi' (time: 1980, lat: 192, lon: 288)>
array([[[-1.29879808e+00, -1.29874921e+00, -1.29880178e+00, ...,
         -1.29877830e+00, -1.29879415e+00, -1.29874623e+00],
        [-6.59829915e-01, -7.32361495e-01, -7.96703756e-01, ...,
         -6.56125963e-01, -5.78027606e-01, -6.67823076e-01],
        [-9.70950246e-01, -1.05041409e+00, -1.12669444e+00, ...,
         -1.02062154e+00, -1.00690091e+00, -9.92955208e-01],
        ...,
        [-1.69000804e+00, -1.68436551e+00, -1.68136072e+00, ...,
         -1.72907245e+00, -1.71857440e+00, -1.70442021e+00],
        [-1.70745599e+00, -1.67744148e+00, -1.64244580e+00, ...,
         -1.73374414e+00, -1.74401677e+00, -1.73630953e+00],
        [-2.27392530e+00, -2.27393675e+00, -2.27395773e+00, ...,
         -2.27400637e+00, -2.27399445e+00, -2.27392220e+00]],

       [[ 2.68323362e-01,  2.68315703e-01,  2.68314511e-01, ...,
          2.68335044e-01,  2.68318295e-01,  2.68335611e-01],
        [ 9.30726767e-01,  9.04935300e-01,  8.77247930e-01, ...,
          8.94442379e-01,  9.17094707e-01,  9.28777039e-01],
        [ 1.00569379e+00,  9.64081287e-01,  9.10039067e-01, ...,
          1.16337597e+00,  1.08726370e+00,  1.06098020e+00],
...
         -4.13265586e-01, -3.88764471e-01, -3.95172626e-01],
        [-2.51653522e-01, -2.47113124e-01, -2.63035446e-01, ...,
         -2.28039920e-01, -2.40550861e-01, -2.50348330e-01],
        [-8.75181109e-02, -8.75648484e-02, -8.75080004e-02, ...,
         -8.73913392e-02, -8.74147788e-02, -8.74597654e-02]],

       [[ 1.35629654e+00,  1.35671806e+00,  1.35641444e+00, ...,
          1.35654974e+00,  1.35602605e+00,  1.35637081e+00],
        [ 1.35223150e+00,  1.34794068e+00,  1.33875740e+00, ...,
          1.35656476e+00,  1.35761344e+00,  1.35010290e+00],
        [ 5.34136891e-01,  5.57700455e-01,  5.47202706e-01, ...,
          6.58094406e-01,  6.17235124e-01,  5.88170290e-01],
        ...,
        [-3.27184528e-01, -3.09917331e-01, -2.93082386e-01, ...,
         -3.30408156e-01, -3.23785514e-01, -3.25003982e-01],
        [-8.55228528e-02, -8.03114027e-02, -7.16066137e-02, ...,
         -1.02032810e-01, -9.24104750e-02, -8.66475627e-02],
        [ 1.19434550e-01,  1.19415157e-01,  1.19466573e-01, ...,
          1.19471639e-01,  1.19577080e-01,  1.19435705e-01]]],
      dtype=float32)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:  Ice Water Path
    units:      kg m-2

In [7]:
##  clwvi
clwvi=  clwvi_ds.clwvi
### Preprocessing
### remove Seasonal Cycle
clwvi_RSC=removeSC(clwvi)
### Detrend
clwvi_RSC_dt=detrend(clwvi_RSC,0)
## Calculate Standard normal anomaly 
FinalOut=calStdNorAnom(clwvi_RSC_dt)
clwvi_Pre_ds = xr.Dataset({'clwvi': (('time','lat','lon'), FinalOut.data,{'long_name':clwvi.long_name, 'units':clwvi.units})}, coords={'time': clwvi.time,'lat': clwvi.lat,'lon': clwvi.lon})
clwvi_Pre_ds.clwvi

<xarray.DataArray 'clwvi' (time: 1980, lat: 192, lon: 288)>
array([[[ 0.12461757,  0.12459242,  0.12455268, ...,  0.12462306,
          0.12460502,  0.12512414],
        [ 0.06302965,  0.03986824,  0.02014108, ...,  0.11750881,
          0.11335173,  0.07865863],
        [ 0.58675176,  0.60983884,  0.64824027, ...,  0.65100485,
          0.6125308 ,  0.5705417 ],
        ...,
        [ 0.65298074,  0.6730923 ,  0.6933681 , ...,  0.574967  ,
          0.5966841 ,  0.62838274],
        [ 1.0706664 ,  1.0764354 ,  1.0179772 , ...,  1.076094  ,
          1.0593159 ,  1.0601432 ],
        [ 0.7295203 ,  0.7294128 ,  0.7294358 , ...,  0.72942024,
          0.72979945,  0.72990626]],

       [[-0.9468839 , -0.94685924, -0.9468613 , ..., -0.94688356,
         -0.9468805 , -0.94687086],
        [-0.814166  , -0.81429285, -0.8135806 , ..., -0.82897586,
         -0.82210183, -0.81696916],
        [-0.7972713 , -0.8055609 , -0.8141113 , ..., -0.7739494 ,
         -0.78493977, -0.7915632 ],
...
        [ 1.6595408 ,  1.6611778 ,  1.6475204 , ...,  1.6643898 ,
          1.6630443 ,  1.6575596 ],
        [ 1.3925496 ,  1.4020947 ,  1.4139276 , ...,  1.3617159 ,
          1.3747734 ,  1.3798525 ],
        [ 1.2062426 ,  1.2048446 ,  1.2042934 , ...,  1.2066463 ,
          1.207357  ,  1.2015901 ]],

       [[-0.31748798, -0.31751242, -0.31752548, ..., -0.31750473,
         -0.31750527, -0.3174915 ],
        [ 0.2248125 ,  0.22104436,  0.22363396, ...,  0.2162747 ,
          0.21884622,  0.22279745],
        [ 0.2808955 ,  0.27783674,  0.28283116, ...,  0.301847  ,
          0.29381338,  0.29026502],
        ...,
        [-1.0898322 , -1.0573775 , -1.0676322 , ..., -1.1083162 ,
         -1.1025219 , -1.0995204 ],
        [-1.3848828 , -1.3726745 , -1.357409  , ..., -1.4196533 ,
         -1.4020463 , -1.3995546 ],
        [-1.4366299 , -1.4378803 , -1.4379504 , ..., -1.43769   ,
         -1.436371  , -1.4360027 ]]], dtype=float32)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:  Condensed Water Path
    units:      kg m-2

In [8]:
##  clt # clt 
## for current version we are using clt instead of low cloud cover
clt=  clt_ds.clt
### Preprocessing
### remove Seasonal Cycle
clt_RSC=removeSC(clt)
### Detrend
clt_RSC_dt=detrend(clt_RSC,0)
## Calculate Standard normal anomaly 
FinalOut=calStdNorAnom(clt_RSC_dt)
clt_Pre_ds = xr.Dataset({'clt': (('time','lat','lon'), FinalOut.data,{'long_name':clt.long_name, 'units':clt.units})}, coords={'time': clt.time,'lat': clt.lat,'lon': clt.lon})
clt_Pre_ds.clt

<xarray.DataArray 'clt' (time: 1980, lat: 192, lon: 288)>
array([[[-1.2042135 , -1.2042135 , -1.2042147 , ..., -1.2042147 ,
         -1.2042181 , -1.2042149 ],
        [-1.3825318 , -1.4890236 , -1.5002258 , ..., -1.3205919 ,
         -1.3455877 , -1.3675636 ],
        [-1.321369  , -1.3777714 , -1.4232026 , ..., -1.1968566 ,
         -1.2404827 , -1.2610903 ],
        ...,
        [-0.5562071 , -0.56235564, -0.6249151 , ..., -0.52881086,
         -0.5350683 , -0.56761444],
        [-0.74227047, -0.73315656, -0.74114996, ..., -0.7070413 ,
         -0.764981  , -0.75212955],
        [-1.8451602 , -1.8451645 , -1.8458488 , ..., -1.8447849 ,
         -1.8447623 , -1.8450018 ]],

       [[ 0.15606655,  0.15606199,  0.15605654, ...,  0.15606745,
          0.15606676,  0.15606566],
        [ 0.948981  ,  0.9497361 ,  0.95577633, ...,  0.9619322 ,
          0.95285743,  0.95331746],
        [ 1.2673327 ,  1.2717354 ,  1.2842586 , ...,  1.2364846 ,
          1.2487968 ,  1.2571762 ],
...
        [-0.27120358, -0.255679  , -0.267617  , ..., -0.30373693,
         -0.3095178 , -0.3111755 ],
        [-0.5711966 , -0.5399673 , -0.5716756 , ..., -0.663215  ,
         -0.63845265, -0.60100865],
        [-0.9287405 , -0.9285001 , -0.92695934, ..., -0.93193763,
         -0.9308057 , -0.92978555]],

       [[-0.4152333 , -0.41523305, -0.41523296, ..., -0.4152315 ,
         -0.4152318 , -0.4152322 ],
        [-0.85068494, -0.84821194, -0.8503579 , ..., -0.81978106,
         -0.8354416 , -0.8382201 ],
        [-0.36770365, -0.38208407, -0.3937147 , ..., -0.31861052,
         -0.33258125, -0.3568672 ],
        ...,
        [-1.0715665 , -1.0872604 , -1.1000744 , ..., -1.0090299 ,
         -1.0354279 , -1.0648097 ],
        [-0.03195607,  0.01928919,  0.09858394, ..., -0.0996014 ,
         -0.11112139, -0.06494474],
        [ 0.3839219 ,  0.38330886,  0.38423502, ...,  0.38302538,
          0.38191023,  0.3825573 ]]], dtype=float32)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:  Total Cloud Cover Percentage
    units:      %

# Output Variables

In [9]:
# psl
##  psl
psl=  psl_ds.psl
### Preprocessing
### remove Seasonal Cycle
psl_RSC=removeSC(psl)
### Detrend
psl_RSC_dt=detrend(psl_RSC,0)
## Calculate Standard normal anomaly 
FinalOut=calStdNorAnom(psl_RSC_dt)
psl_Pre_ds = xr.Dataset({'psl': (('time','lat','lon'), FinalOut.data, {'long_name':psl.long_name, 'units':psl.units})}, coords={'time': psl.time,'lat': psl.lat,'lon': psl.lon})
psl_Pre_ds.psl

<xarray.DataArray 'psl' (time: 1980, lat: 192, lon: 288)>
array([[[-0.47794086, -0.47794083, -0.4779414 , ..., -0.4779422 ,
         -0.4779414 , -0.47794086],
        [-0.49899426, -0.49910167, -0.49913993, ..., -0.4990875 ,
         -0.49893737, -0.49896735],
        [-0.54701924, -0.54744375, -0.5478834 , ..., -0.5464757 ,
         -0.5466395 , -0.5465874 ],
        ...,
        [ 2.5952094 ,  2.5900571 ,  2.584817  , ...,  2.6099143 ,
          2.6051986 ,  2.6002889 ],
        [ 2.490274  ,  2.4876862 ,  2.4850724 , ...,  2.4979289 ,
          2.4954169 ,  2.492852  ],
        [ 2.374492  ,  2.374492  ,  2.374492  , ...,  2.374492  ,
          2.374492  ,  2.374492  ]],

       [[-0.6666005 , -0.6666005 , -0.6666006 , ..., -0.6666007 ,
         -0.66660064, -0.6665994 ],
        [-0.6565732 , -0.6567747 , -0.65717906, ..., -0.6557439 ,
         -0.65598184, -0.65624017],
        [-0.6465372 , -0.64616394, -0.64572555, ..., -0.64924574,
         -0.6480923 , -0.647181  ],
...
        [-0.03862502, -0.03616651, -0.03387383, ..., -0.04647383,
         -0.04384604, -0.04119873],
        [-0.09124327, -0.09047402, -0.08976924, ..., -0.09426039,
         -0.09313117, -0.09212784],
        [-0.194553  , -0.194553  , -0.194553  , ..., -0.194553  ,
         -0.194553  , -0.194553  ]],

       [[ 0.6951347 ,  0.69513446,  0.6951347 , ...,  0.6951323 ,
          0.6951331 ,  0.6951343 ],
        [ 0.7234464 ,  0.7247392 ,  0.7261365 , ...,  0.7187446 ,
          0.72041535,  0.7219753 ],
        [ 0.7589059 ,  0.759741  ,  0.76058793, ...,  0.7552556 ,
          0.75665444,  0.7579078 ],
        ...,
        [-0.13859382, -0.14310391, -0.14771056, ..., -0.1254692 ,
         -0.12977304, -0.13416168],
        [-0.16849042, -0.1690478 , -0.16952862, ..., -0.16619112,
         -0.16704004, -0.16780424],
        [-0.15908894, -0.15908894, -0.15908894, ..., -0.15908894,
         -0.15908894, -0.15908894]]], dtype=float32)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:  Sea Level Pressure
    units:      Pa

In [10]:
##  pr
pr=  pr_ds.pr
### Preprocessing
### remove Seasonal Cycle
pr_RSC=removeSC(pr)
### Detrend
pr_RSC_dt=detrend(pr_RSC,0)
## Calculate Standard normal anomaly 
FinalOut=calStdNorAnom(pr_RSC_dt)
pr_Pre_ds = xr.Dataset({'pr': (('time','lat','lon'), FinalOut.data,{'long_name':pr.long_name, 'units':pr.units})}, coords={'time': pr.time,'lat': pr.lat,'lon': pr.lon})
pr_Pre_ds.pr

<xarray.DataArray 'pr' (time: 1980, lat: 192, lon: 288)>
array([[[-1.2548518 , -1.2548507 , -1.25485   , ..., -1.254837  ,
         -1.2548519 , -1.254848  ],
        [-0.7800134 , -0.7535704 , -0.7438767 , ..., -0.89213306,
         -0.8650806 , -0.8025866 ],
        [-0.9580544 , -1.0040976 , -1.055458  , ..., -1.0251942 ,
         -0.9821431 , -0.96126014],
        ...,
        [-1.033839  , -1.0555588 , -1.074983  , ..., -0.9386782 ,
         -0.97626376, -1.0090852 ],
        [-0.75391334, -0.7687472 , -0.78117937, ..., -0.7122136 ,
         -0.72335   , -0.7378924 ],
        [-1.2322668 , -1.2323346 , -1.2323598 , ..., -1.231967  ,
         -1.2320482 , -1.232176  ]],

       [[ 0.40784305,  0.40783373,  0.4078357 , ...,  0.40784448,
          0.4078449 ,  0.4078434 ],
        [ 0.80522394,  0.80597687,  0.79743886, ...,  0.83363223,
          0.8227532 ,  0.8099578 ],
        [ 0.9340124 ,  0.89995086,  0.86227834, ...,  1.0181736 ,
          0.99119025,  0.9608124 ],
...
        [ 2.1467586 ,  2.1559505 ,  2.1656463 , ...,  2.1242175 ,
          2.1285768 ,  2.1379771 ],
        [ 2.2412562 ,  2.2523944 ,  2.2652442 , ...,  2.215693  ,
          2.2196383 ,  2.2271996 ],
        [ 1.8467216 ,  1.8465946 ,  1.8467146 , ...,  1.8466152 ,
          1.846223  ,  1.8462132 ]],

       [[ 1.067249  ,  1.067241  ,  1.0672171 , ...,  1.0672176 ,
          1.0672079 ,  1.067246  ],
        [ 1.0630687 ,  1.06463   ,  1.0633657 , ...,  1.0602413 ,
          1.0614876 ,  1.0623307 ],
        [ 0.745816  ,  0.7212167 ,  0.6859137 , ...,  0.86964107,
          0.8238499 ,  0.78205305],
        ...,
        [-0.32582307, -0.31369367, -0.3011431 , ..., -0.35080206,
         -0.34476918, -0.3357236 ],
        [-0.4342615 , -0.42461172, -0.414908  , ..., -0.46011952,
         -0.4516764 , -0.44368   ],
        [-0.34342808, -0.34333476, -0.34330463, ..., -0.34340036,
         -0.34348324, -0.34342864]]], dtype=float32)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:  Precipitation
    units:      kg m-2 s-1

In [11]:
##  tas
tas=  tas_ds.tas
### tasetasocessing
### remove Seasonal Cycle
tas_RSC=removeSC(tas)
### Detrend
tas_RSC_dt=detrend(tas_RSC,0)
## Calculate Standard normal anomaly 
FinalOut=calStdNorAnom(tas_RSC_dt)
tas_Pre_ds = xr.Dataset({'tas': (('time','lat','lon'), FinalOut.data,{'long_name':tas.long_name, 'units':tas.units})}, coords={'time': tas.time,'lat': tas.lat,'lon': tas.lon})
tas_Pre_ds.tas

<xarray.DataArray 'tas' (time: 1980, lat: 192, lon: 288)>
array([[[-0.03881278, -0.03880946, -0.03880598, ..., -0.03880375,
         -0.03880692, -0.03881099],
        [-0.02385203, -0.02413527, -0.02581471, ..., -0.01554035,
         -0.01830069, -0.02258492],
        [ 0.01696229,  0.01741131,  0.01768197, ...,  0.0280712 ,
          0.02577301,  0.01888501],
        ...,
        [ 1.7348824 ,  1.7468804 ,  1.7584449 , ...,  1.7090676 ,
          1.7170072 ,  1.7247261 ],
        [ 1.6429307 ,  1.6446201 ,  1.6464292 , ...,  1.6359717 ,
          1.6386571 ,  1.640957  ],
        [ 1.5192255 ,  1.5199908 ,  1.520684  , ...,  1.5164278 ,
          1.5174558 ,  1.5183805 ]],

       [[-0.9045083 , -0.90450794, -0.9045078 , ..., -0.9045168 ,
         -0.9045164 , -0.90451854],
        [-0.8439341 , -0.8444455 , -0.8439151 , ..., -0.8476321 ,
         -0.8463678 , -0.84545505],
        [-0.7956598 , -0.7964528 , -0.7991637 , ..., -0.7862967 ,
         -0.79043275, -0.7935608 ],
...
        [ 0.5234003 ,  0.5218432 ,  0.5199899 , ...,  0.52541494,
          0.5248566 ,  0.52426517],
        [ 0.51806253,  0.5193728 ,  0.5207327 , ...,  0.513785  ,
          0.5154366 ,  0.5167825 ],
        [ 0.57135135,  0.5710964 ,  0.5708669 , ...,  0.57227093,
          0.5719312 ,  0.57162213]],

       [[ 0.6919365 ,  0.69193923,  0.69193894, ...,  0.69193476,
          0.69193476,  0.6919346 ],
        [ 0.68877834,  0.6912362 ,  0.69247615, ...,  0.68010676,
          0.68197244,  0.68574363],
        [ 0.6170692 ,  0.62088895,  0.6308948 , ...,  0.60962266,
          0.61185914,  0.6136613 ],
        ...,
        [-1.2214682 , -1.2207301 , -1.2197009 , ..., -1.2221763 ,
         -1.2222067 , -1.2218016 ],
        [-1.1819122 , -1.1793109 , -1.1764619 , ..., -1.1886551 ,
         -1.1866935 , -1.1843868 ],
        [-1.0673071 , -1.0678486 , -1.0683355 , ..., -1.0653253 ,
         -1.0660534 , -1.0667114 ]]], dtype=float32)
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    long_name:  Near-Surface Air Temperature
    units:      K

# Input and Output 
Finally combine all dataArrays as a dataset

In [12]:
#### Input 
Processed_CESM2_r1i1p1f1_historical_Input = xr.merge([cres_Pre_ds.cres,alpha_Pre_ds.alpha,clt_Pre_ds.clt,clwvi_Pre_ds.clwvi,clivi_Pre_ds.clivi],combine_attrs="drop_conflicts")
Processed_CESM2_r1i1p1f1_historical_Input.to_netcdf(path='Processed_CESM2_r1i1p1f1_historical_Input.nc',mode='w',format='NETCDF4')
### Output
Processed_CESM2_r1i1p1f1_historical_Output = xr.merge([tas_Pre_ds.tas,psl_Pre_ds.psl,pr_Pre_ds.pr],combine_attrs="drop_conflicts")

Processed_CESM2_r1i1p1f1_historical_Output.to_netcdf(path='Processed_CESM2_r1i1p1f1_historical_Output.nc',mode='w',format='NETCDF4')